In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import pickle as pkl
from collections import defaultdict
import viz
import sys
import notebook_helper
from matplotlib.backends.backend_pdf import PdfPages
sys.path.append('../experiments/')
# results_dir = '/home/chansingh/mntv1/tree-prompt/feb25'
# results_dir = '/home/chansingh/mntv1/tree-prompt/mar17'
results_dir = '/home/chansingh/mntv1/tree-prompt/mar18'

r = imodelsx.process_results.get_results_df(results_dir)

# fill missing args with default values from argparse
experiment_filename = '../experiments/01_fit.py'
r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename)
r['mean_llm_calls'] = notebook_helper.add_mean_llm_calls(r)

# group using these experiment hyperparams when averaging over random seeds
group_keys = [
    k for k in imodelsx.process_results.get_main_args_list(experiment_filename)
    if not k == 'seed'
]
ravg = (
    r
    .groupby(by=group_keys)
    .mean(numeric_only=True)
    .reset_index()
)

In [ ]:
groupings = ['checkpoint', 'prompt_source', 'verbalizer_num'] #, 'binary_classification']
for k, rcurve in ravg.groupby(groupings):
    rcurve = rcurve.sort_values(by=['dataset_name', 'model_name', 'mean_llm_calls'])

    metric = 'roc_auc'
    # metric = 'accuracy'
    x = 'mean_llm_calls'                # x = 'num_prompts'

    viz.plot_perf_curves_individual(rcurve, x=x, xlim=15)
    x = rcurve.iloc[0]
    plt.suptitle(f'''checkpoint={viz.CHECKPOINTS_RENAME_DICT[x.checkpoint]} prompt_source={x.prompt_source} verb={x.verbalizer_num}''', fontsize='xx-small')
    # plt.tight_layout()
    # plt.show()

viz.save_figs_to_single_pdf("all_results.pdf")

### Apply cross-validation

In [ ]:
# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='accuracy_cv', ascending=False)
    .groupby(by=['split_strategy', 'dataset_name'])
    .first()
    .reset_index()
)
ravg_cv

### Look at values

In [16]:
groupings = ['checkpoint', 'prompt_source', 'verbalizer_num']
dset_names = ravg.dataset_name.unique().tolist()
mets = defaultdict(list)
for k, rcurve in ravg.groupby(groupings):
    row = rcurve.iloc[0]
    rcurve = rcurve.sort_values(by='accuracy_cv', ascending=False)
    for kk in groupings:
        mets[kk].append(row[kk])
    metric = 'roc_auc'
    x = 'mean_llm_calls'
pd.DataFrame.from_dict(mets)

,checkpoint,prompt_source,verbalizer_num
0,EleutherAI/gpt-j-6B,data_demonstrations,0
1,EleutherAI/gpt-j-6B,data_demonstrations,1
2,EleutherAI/gpt-j-6B,manual,0
3,gpt2,data_demonstrations,0
4,gpt2,data_demonstrations,1
5,gpt2,manual,0
